## Projet 4
***
### Réaliser une étude de santé publique avec Python





*Nous nous donnons pour mission de faire un état des lieu de la malnutrition dans le monde pour l'année 2017 en répondant aux interrogations suivantes*

   - Quelle est la proportion de personnes en état de sous-nutrition ?
   - Quel est le nombre théorique de personnes qui pourraient être nourries ? 
   - Comment est utilisée la disponibilité alimentaire intérieure dans chaque pays ?
   - Quels sont les pays qui souffrent le plus de sous nutrition ?
   - Quels pays bénéficient le plus de l'aide internationale ? 
   - Quelle est la place des céréales dans l'alimentation ?
   - Quel est la place de l'exportation par rapport à la production ?

*À cette fin, nous utilisons les données provenant du site de la FAO (Food and Agriculture Organization) que nous allons exploiter comme suit :*





### Sommaire :
***

### <a href="#C1"><font color='Black'>Partie 1 : Préparation des données</font></a>
   - <a href="#C2">Importation des librairies</a>
   - <a href="#C3">Importation des données</a>
   - <a href="#C4">Traitement des données</a>
        - <a href="#C5">1) Population</a>
        - <a href="#C6">2) Sous nutrition</a>
        - <a href="#C7">3) Aide alimentaire</a>
        - <a href="#C8">4) Disponibilité alimentaire</a>
***


### <a href="#C9"><font color='Black'>Partie 2 : Analyse des données</font></a>

   - <a href="#C10">1) Proportion de personnes en état de sous-nutrition</a>
   - <a href="#C11">2) Nombre théorique de personnes qui pourraient être nourries</a> 
   - <a href="#C12">3) Nombre théorique de personnes qui pourraient être nourries exclusivement avec des produits végétaux</a> 
   - <a href="#C13">4) Utilisation de la disponibilité alimentaire intérieure</a>
   - <a href="#C14">5) Pays pour lesquels la proportion de personnes sous-nutrition est la plus forte</a>
   - <a href="#C15">6) Pays qui ont le plus bénéficié d’aide depuis 2013</a>
   - <a href="#C16">7) Disponibilité par habitant dans chaque pays</a>
   - <a href="#C17">8) Céréales dans l'alimentation</a> 
   - <a href="#C18">9) Exemple du manioc en Thaïlande</a>

## **<a name="C1"><font color='Black'>Partie 1 : Préparation des données</font></a>**
***
### <a name="C2"><font color='Black'>Importation des librairies :</font></a>
***
Les librairies que nous utiliserons sont les suivantes :
   - Numpy pour les tableaux et les opérations  
   - Pandas pour la manipulation des données
   - Matplotlib pour tracer des representations graphiques
   - Seaborn pour améliorer les graphiques

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### <a name="C3"><font color='Black'>Importation des données</font></a>
***
Les fichiers de données à notre disposition sont les suivants :
   - **population.csv** : population par pays 
   - **sous_nutrition.csv** : Nombre de personne en sous-alimentation par pays
   - **aide_alimentaire.csv** : quantité d'aide alimentaire par pays
   - **dispo_alimentaire.csv** : disponibilité de diverses denrées alimentaires par pays

In [5]:
population = pd.read_csv('population.csv')
dispo_alimentaire = pd.read_csv('dispo_alimentaire.csv')
sous_nutrition = pd.read_csv('sous_nutrition.csv')
aide_alimentaire = pd.read_csv('aide_alimentaire.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'population.csv'

### <a name="C4"><font color='Black'>Traitement des données</font></a>
***
Avant de préparer nos données, nous devons obtenir une description exacte de ces dernières :
   - le nombre de lignes et de colonnes de chaque jeu de données
   - un dénombrage des valeurs totales et uniques
   - le type de chaque colonne
   - un dénombrage des valeurs nulles
   - un dénombrage des valeurs aberrantes

Pour cela, nous créons une fonction qui retourne un tableau comprenant toutes les informations dont nous avons besoin :

In [ ]:
def info_donnee(df: pd.DataFrame):
    desc = df.columns.to_frame(name="colonne").set_index('colonne')
    desc['nombre de valeurs non nulles'] = df.notnull().sum() 
    desc['nombre de valeurs uniques'] = df.nunique() 
    desc['type de donnée'] = df.dtypes 
    desc['nombre de valeurs nulles']=df.isna().sum()
    desc['pourcentage de valeurs nulles']=round((df.isna().sum()/df.shape[0]*100),2)
    return desc

### <a name="C5"><font color='Black'>1) Population</font></a>
***
#### Description
Le fichier population inclut :
   - Le pays
   - L’année
   - La population du pays, exprimée en milliers d’habitants

Nous utilisons la fonction créée plus haut afin d'examiner le jeu de données :

In [ ]:
info_donnee(population).reset_index()

Nous constatons qu'il n'y a pas de valeurs manquantes, 1416 valeurs pour chaque colonne.

#### Modification

Voici les modifications que nous allons apporter sur ce dataset :
   - renommage des variables
   - conversion de la population en unité réélle
   - conversion de la colonne habitants en entier


In [ ]:
population = population.rename(columns={
    'Zone'  : 'pays', 
    'Année' : 'annee', 
    'Valeur': 'nombre_habitants'})

population['nombre_habitants'] = population['nombre_habitants'] * 1000

population['nombre_habitants'] = population['nombre_habitants'].astype(int)

population.head()

In [ ]:
info_donnee(population).reset_index()

### <a name="C6"><font color='Black'>2) Sous nutrition</font></a>
***
#### Description
Le fichier d’insécurité alimentaire inclut :
   - Le pays
   - Intervalle d’années
   - Le nombre de personnes en sous-alimentation en millions d’habitants 

Nous utilisons la fonction créée plus haut afin d'examiner le jeu de données :

In [ ]:
info_donnee(sous_nutrition).reset_index()

Nous constatons qu'il y a des valeurs manquantes dans la colonne "valeur"

#### Modification

Voici les modifications que nous allons apporter sur ce dataset :
   - renommage des variables
   - transformation des valeurs nulles en "0"
   - suppression des caractères spéciaux "<"
   - simplifier les dates pour avoir la moyenne plutot qu'un interval
   - conversion de la colonne "valeur" en int
   - conversion de la population en unité réélle


In [ ]:
sous_nutrition = sous_nutrition.rename(columns={
    'Zone'  : 'pays', 
    'Année' : 'annee', 
    'Valeur': 'habitants_sous_nutrition'})

sous_nutrition['habitants_sous_nutrition'].fillna(0, inplace=True)

sous_nutrition.loc[ sous_nutrition['habitants_sous_nutrition'] == '<0.1', 'habitants_sous_nutrition' ] = 0

sous_nutrition['annee'] = sous_nutrition['annee'].replace(
    ['2012-2014','2013-2015','2014-2016','2015-2017','2016-2018','2017-2019'],
    [   '2013',     '2014',     '2015',     '2016',     '2017',     '2018'])

sous_nutrition['annee'] = sous_nutrition['annee'].astype(np.int64)

sous_nutrition['habitants_sous_nutrition'] = sous_nutrition['habitants_sous_nutrition'].astype(float, errors = 'ignore')

sous_nutrition['habitants_sous_nutrition'] = sous_nutrition['habitants_sous_nutrition']*1000000

sous_nutrition['habitants_sous_nutrition'] = sous_nutrition['habitants_sous_nutrition'].astype(int, errors = 'ignore')


sous_nutrition.head()

In [ ]:
info_donnee(sous_nutrition).reset_index()

### <a name="C7"><font color='Black'>3) Aide alimentaire</font></a>
***
#### Description
Le fichier d’aide alimentaire inclut :
   - Le pays bénéficiaire de l'aide
   - L’année
   - Le produit
   - La quantité d'aide alimentaire donnée en tonnes 

Nous utilisons la fonction créée plus haut afin d'examiner le jeu de données :

In [ ]:
info_donnee(aide_alimentaire).reset_index()

Nous constatons qu'il n'y a pas de valeurs manquantes, 1475 valeurs pour chaque colonne.

#### Modification

Voici les modifications que nous allons apporter sur ce dataset :
   - renommage des variables
   - conversion de la quantité d'aide alimentaire en kilogramme


In [ ]:
aide_alimentaire = aide_alimentaire.rename(columns={
    'Pays bénéficiaire': 'pays', 
    'Année'            : 'annee', 
    'Produit'          : 'produit', 
    'Valeur'           : 'quantite_aide_kg'})

aide_alimentaire['quantite_aide_kg'] = aide_alimentaire['quantite_aide_kg'] * 1000


aide_alimentaire.head()

### <a name="C8"><font color='Black'>4) Disponibilité alimentaire</font></a>
***
#### Description
Le fichier de disponibilité alimentaire inclut :
   - Le pays
   - Le produit
   - 4 variables de disponibilité alimentaire
   - 10 variables qui précisent les utilisations/moyens d’acquisition de la disponibilité alimentaire en milliers de tonnes
   - Une variable type qui précise si le produit est d’origine animale ou végétale

Nous utilisons la fonction créée plus haut afin d'examiner le jeu de données :

In [ ]:
info_donnee(dispo_alimentaire).reset_index()

Nous constatons qu'il y a de nombreues valeurs manquantes dans plusieurs colonnes, les unités utilisées sont incohérentes.

#### Modification

Voici les modifications que nous allons apporter sur ce dataset :
   - renommage des variables
   - transformation des valeurs nulles en "0"
   - conversion des variables en kilogramme

In [ ]:
dispo_alimentaire = dispo_alimentaire.rename(columns={
    "Zone"                                                         : "pays", 
    "Produit"                                                      : "produit", 
    "Origine"                                                      : "origine_produit", 
    "Aliments pour animaux"                                        : "aliments_pour_animaux", 
    "Autres Utilisations"                                          : "autres_utilisations", 
    "Disponibilité alimentaire (Kcal/personne/jour)"               : "dispo_alim_kcal_pers_j", 
    "Disponibilité alimentaire en quantité (kg/personne/an)"       : "dispo_alim_kg_pers_an", 
    "Disponibilité de matière grasse en quantité (g/personne/jour)": "dispo_mat_grasse_gr_pers_j", 
    "Disponibilité de protéines en quantité (g/personne/jour)"     : "dispo_proteine_gr_pers_j", 
    "Disponibilité intérieure"                                     : "dispo_interieure", 
    "Exportations - Quantité"                                      : "quantite_exportation", 
    "Importations - Quantité"                                      : "quantite_importation", 
    "Nourriture"                                                   : "quantite_nourriture", 
    "Pertes"                                                       : "pertes_nourriture", 
    "Production"                                                   : "production_nourriture", 
    "Semences"                                                     : "semences", 
    "Traitement"                                                   : "traitement", 
    "Variation de stock"                                           : "variation_stock"})

dispo_alimentaire.fillna(0, inplace=True)

dispo_alimentaire['dispo_alim_kg_pers_an'] = dispo_alimentaire['dispo_alim_kg_pers_an'] * 1000000
dispo_alimentaire['dispo_interieure'] = dispo_alimentaire['dispo_interieure'] * 1000000
dispo_alimentaire['quantite_nourriture'] = dispo_alimentaire['quantite_nourriture'] * 1000000
dispo_alimentaire['aliments_pour_animaux'] = dispo_alimentaire['aliments_pour_animaux'] * 1000000
dispo_alimentaire['pertes_nourriture'] = dispo_alimentaire['pertes_nourriture'] * 1000000
dispo_alimentaire['semences'] = dispo_alimentaire['semences'] * 1000000
dispo_alimentaire['traitement'] = dispo_alimentaire['traitement'] * 1000000
dispo_alimentaire['autres_utilisations'] = dispo_alimentaire['autres_utilisations'] * 1000000

dispo_alimentaire.head()

## **<a name="C9"><font color='Black'>Partie 2 : Analyse des données</font></a>**
***
Nos données sont desormais nettoyées et homogénéisées, nous allons procéder à l'analyse de ces dernières. Nous traiterons d'abord les demandes de Marc puis celle de Mélanie, enfin nous traiterons les demandes subsidiaires de Julien. Pour chaque demande nous décrirons la méthode avant de l'appliquer, et nous apporterons une conclusion. Nous partons du postulat que toutes les données doivent êtres analysées pour l'année 2017 uniquement sauf quand une date est précisée.

### <a name="C10"><font color='Black'>1) Proportion de personnes en état de sous-nutrition</font></a>
***
#### Démarche
   - extraction de la population de 2017
   - somme de la population de chaque pays
   - extraction de la population en sous nutrition
   - somme de la population en sous nutrition
   - rapport entre les 2 données pour obtenir une proportion

In [ ]:
population_2017 = population[(population["annee"] == 2017)]

population_totale_2017 = population_2017['nombre_habitants'].sum()

print("La population mondiale est de ",population_totale_2017," individus.")

sous_nutrition_2017 = sous_nutrition.loc[sous_nutrition['annee'] == 2017]

sous_nutrition_mondiale_2017 = sous_nutrition_2017['habitants_sous_nutrition'].sum()

print("La population en sous nutrition est de ",sous_nutrition_mondiale_2017," individus")

proportion_sous_nutrition_2017 = (sous_nutrition_mondiale_2017 / population_totale_2017) * 100

print( proportion_sous_nutrition_2017, "% de la population était en sous nutrition")


#### Conclusion
D'après les données de la FAO, 7.1 % de la population mondiale était en état de sous nutrition en 2017.

###  <a name="C11"><font color='Black'>2) Nombre théorique de personnes qui pourraient être nourries</font></a>  
***
Postulat : D'après l'Organisation mondiale de la Santé, un homme doit consommer environ 2 500 kilocalories (kcal) par jour, contre 2 000 kcal/jour pour une femme. La Terre étant peuplée de femmes et d'hommes à quasiment 50/50, nous considérons une moyenne de besoin de 2250 kcal/jour par individu.
#### Démarche
   - fusion des dataframes dispo_alimentaire et population_2017 sur la clé "pays"
   - calcul de la disponibilité en kcal (dispo par pers * nb d'habitants)
   - déclaration de la variable besoin_kcal_pers_j 
   - division de la dispo alimentaire totale en kcal par le besoin journalier par individu
   - calcul de la  proportion en %

In [ ]:
dispo_alimentaire_2017 = pd.merge(dispo_alimentaire, population_2017, on = 'pays')

dispo_alim_kcal_2017 = dispo_alimentaire_2017['dispo_alim_kcal_pers_j'] * dispo_alimentaire_2017['nombre_habitants']

besoin_kcal_pers_j = 2250

nb_personne_pouvant_etre_nourries = dispo_alim_kcal_2017.sum() / besoin_kcal_pers_j

pourcentage_pouvant_etre_nourries = (nb_personne_pouvant_etre_nourries * 100) / population_2017['nombre_habitants'].sum() 

print( 'Le nombre de personnes pouvant etre nourries est', nb_personne_pouvant_etre_nourries,' soit ', pourcentage_pouvant_etre_nourries, '% de la population totale')



#### Conclusion
D'après les données de la FAO, nous estimons qu'environ 9,2 Milliard de personnes auraient pu être nourries avec les disponibilités alimentaires de l'année 2017, ce qui représente 123% de la population.  

###  <a name="C12"><font color='Black'>3) Nombre théorique de personnes qui pourraient être nourries exclusivement avec des produits végétaux</font></a>
***
#### Démarche

   - extraction de la disponibilité alimentaire végétale en séléctionnant l'origine du produit = vegetale
   - calcul de la disponibilité en kcal (dispo par pers * nb d'habitants)
   - division de la dispo alimentaire végétale totale en kcal par le besoin journalier par individu
   - calcul de la proportion en %

In [ ]:
dispo_alimentaire_vegetale = dispo_alimentaire_2017[dispo_alimentaire_2017['origine_produit'] == "vegetale"]

dispo_alimentaire_vegetale = dispo_alimentaire_vegetale['dispo_alim_kcal_pers_j'] * dispo_alimentaire_2017['nombre_habitants']

nb_personne_pouvant_etre_nourries_vegetaux = dispo_alimentaire_vegetale.sum() / besoin_kcal_pers_j

pourcentage_personne_pouvant_etre_nourries_vegetaux = (nb_personne_pouvant_etre_nourries_vegetaux * 100) / population_2017['nombre_habitants'].sum() 

print( 'Le nombre de personnes pouvant etre nourries est', nb_personne_pouvant_etre_nourries_vegetaux, ' soit ', pourcentage_personne_pouvant_etre_nourries_vegetaux, '% de la population totale')


#### Conclusion
En réalisant la même démarche que précedemment, nous estimons qu'environ 7,6 Milliard de personnes auraient pu être nourries avec les disponibilités alimentaires végétales de l'année 2017, ce qui représente 101% de la population. La population aurait donc pu être nourrie exclusivement avec des végétaux.

### <a name="C13"><font color='Black'>4) Utilisation de la disponibilité alimentaire intérieure</font></a>

***
Postulat : la disponibilité intérieure correspond à la disponibilité totale d’un aliment au sein du pays. dans notre dataset, elle se retrouve des 2 façons suivantes : 
   - dispo_interieure = production_nourriture + quantite_importation - quantite_exportation - variation_stock
   - dispo_interieure = aliments_pour_animaux + autres_utilisations + quantite_nourriture + pertes_nourriture + semences + traitement
   
#### Démarche

   - sélection de chaque type d'utilisation de la disponibilité intérieure en regroupant d'abord par pays
   - somme de la disponibilité de tous les pays en conservant chaque type d'utilisation
   - visualisation via un diagramme circulaire des types d'utilisation
   

In [ ]:
disponibilite_interieure = dispo_alimentaire_2017.loc[:, ['pays','aliments_pour_animaux', 'autres_utilisations','quantite_nourriture', 'pertes_nourriture','semences', 'traitement']].groupby('pays').sum()

disponibilite_interieure = disponibilite_interieure.sum()
 
disponibilite_interieure.plot.pie(figsize=(10, 8),autopct='%1.1f%%', startangle=18)
plt.title('Utilisation de la disponibilité alimentaire interieure mondiale')
plt.ylabel("")
plt.show()

#### Conclusion
Nous constatons que moins de 50% de la nourriture produite est directement utilisée pour l'alimentation humaine. Aussi les pertes représentent 4,6% de la disponibilité alimentaire, une part qui pourrait nourrir une quantité non négligeable d'individus.
### <a name="C14"><font color='Black'>5) Pays pour lesquels la proportion de personnes en sous-nutrition est la plus forte</font></a>

***
#### Démarche
   - fusion des dataframes sous-nutrition et population_2017 
   - calcul de la proportion de sous nutrition par pays (habitants en sous nutrition * 100 / nombre d'habitants)
   - classement décroissant des proportions
   - remise à zéro de l'index pour plus de clarté

In [ ]:
sous_nutrition_2017 = pd.merge(sous_nutrition, population_2017, on=['pays', 'annee'], how='inner')

sous_nutrition_2017["pourcentage_sous_nutrition"] = (sous_nutrition_2017["habitants_sous_nutrition"]*100 / sous_nutrition_2017["nombre_habitants"])

sous_nutrition_2017 = sous_nutrition_2017.sort_values(by=["pourcentage_sous_nutrition"], ascending=False)

sous_nutrition_2017 = sous_nutrition_2017.loc[:, ['pays','pourcentage_sous_nutrition']].reset_index(drop=True)

sous_nutrition_2017.head(10)


#### Conclusion
Les populations en sous nutrition atteignent des proportions très importantes, quasiment la moitié de la population de Haïti souffre de sous nutrition, suivi de près par la Corée du Nord et Madagascar. Nous retrouvons aussi dans le haut du classement des pays instables comme le Tchad ou le Rwanda, qui sont empétrés dans des conflits intérieurs depuis de nombreuses années.
### <a name="C15"><font color='Black'>6) Pays ayant le plus bénéficié d’aide depuis 2013</font></a>

***
#### Démarche
   - création d'un tableau qui regroupe la somme des quantités d'aide par pays
   - classement décroissant du total des aides alimentaires
   - ajout d'un index pour plus de clarté

In [ ]:
tableau_aide = aide_alimentaire[['pays','quantite_aide_kg']].groupby('pays').sum()

tableau_aide = tableau_aide.sort_values(by='quantite_aide_kg',ascending=False).reset_index()

tableau_aide.head(10)


#### Conclusion
Nous visualisons un tableau présentant les quantités d'aide distribuées par pays depuis 2013. 
La Syrie tient la tête de ce classement avec 1,8 Million de Tonnes d'aide alimentaire reçue dans cette période, cela coincide avec le conflit Syrien qui a débuté en 2011 et qui n'a toujours pas trouvé d'issue. 
De la même manière, nous constatons que les pays qui occupent la tête de ce classement sont pour la plupart empetrés dans des conflits éxterieurs ou intérieurs. 
Enfin nous notons que les pays qui ont reçu le plus d'aide sont différents des pays qui souffrent le plus de sous nutrition. 
### <a name="C16"><font color='Black'>7) La disponibilité par habitant dans chaque pays</font></a>

***
#### Démarche
   - création d'un dataframe par l'agrégation des disponibilités alimentaires par personne
   - classement des données totales décroissant
   - ajout d'un index pour plus de clarté

In [ ]:
tableau_dispo = dispo_alimentaire[['pays','produit','dispo_alim_kcal_pers_j']].groupby('pays').sum()

tableau_dispo = tableau_dispo.sort_values(by='dispo_alim_kcal_pers_j').reset_index()

tableau_dispo.head(10)

#### Conclusion
Le classement des pays ayant la plus forte disponibilité alimentaire par habitant est trusté par des pays d'europe, riche et stables. Avec en tête l'autriche ou la belgique.
Avec la disponibilité alimentaire par habitant la moins élevée, nous retrouvons le Centrafrique, mais aussi Haïti, la Corée du Nord et le Tchad qui apparaissent aussi dans le classement des pays qui souffrent le plus de sous nutrition.
Il y a évidemment un lien direct entre disponibilité alimentaire et sous nutrition.
###  <a name="C17"><font color='Black'>8) Utilisation des céréales dans l'alimentation humaine et animale</font></a>

***
Postulat : La FAO indique une liste d'aliments considérés comme des céréales, nous utiliserons cette liste de produits pour répondre à cette demande : Le blé, le riz, l'orge, le maïs, le seigle, l'avoine, le millet, et le sorgho.

#### Démarche
   - création d'une liste de céréales pour simplifier la suite
   - vérification de la présence d'une céréale dans la colonne produit avec isin
   - calcul de la proportion d'utilisation animale des céréales
   - calcul de la proportion d'utilisation humaine des céréales

In [ ]:
cereales_fao =  ['Blé et produits', 'Riz et produits', 'orge et produits', 'Maïs et produits', 'Seigle et produits','Avoine', 'Millet et produits', 'Sorgho et produits', 'Céréales, Autres']

table_cereales = dispo_alimentaire.loc[dispo_alimentaire['produit'].isin(cereales_fao),:]

proportion_alimentation_animale = table_cereales['aliments_pour_animaux'].sum() * 100 / table_cereales['dispo_interieure'].sum()

proportion_alimentation_humaine = table_cereales['quantite_nourriture'].sum() * 100 / table_cereales['dispo_interieure'].sum()

print("La proportion d'alimentation animale est", proportion_alimentation_animale)
      
print("La proportion d'alimentation humaine est", proportion_alimentation_humaine)


#### Conclusion
D'après les données fournies par la FAO, nous estimons que 69% des céréales disponibles dans le monde sont déstinées à l'alimentation animale, tandis que seulement 18% des céréales est directement déstiné à l'alimentation humaine.

###  <a name="C18"><font color='Black'>9) Exemple du manioc en Thaïlande</font></a>

***

#### Démarche
   - extraction des informations sur la Thaïlande et sur le manioc avec .loc et &
   - calcul de la proportion d'exportation du manioc
   - extraction de la proportion de sous nutrituion en Thaïlande via la table sous_nutrition_2017

In [ ]:
dispo_manioc_thailande = dispo_alimentaire.loc[(dispo_alimentaire['produit'] == "Manioc") 
                                               & (dispo_alimentaire['pays'] == "Thaïlande"), :]

proportion_exportation_manioc = dispo_manioc_thailande['quantite_exportation'].sum() * 100 / dispo_manioc_thailande['production_nourriture'].sum()

print('La Thaïlande exporte',proportion_exportation_manioc, '% de sa production de manioc')

sous_nutrition_thailande = sous_nutrition_2017.loc[sous_nutrition_2017['pays'] == "Thaïlande"]

sous_nutrition_thailande


#### Conclusion
La Thaïlande exporte plus de 83% de sa production de manioc malgré qu'une part non négligeable de sa population souffre de sous nutrition. En effet, la sous nutrition atteint environ 9% de la population.

#### Démarche 2
   - déclaration de la variable kcal par kg de manioc 
   - extraction de la population de la Thailande en 2017
   - extraction de la quantité d'exportation de manioc en kg
   - calcul quantité kcal disponibles en exportation
   - calcul du nombre de personne pouvant être potentiellement nourries par le manioc en export
   - calcul de ce nombre en proportion de la population


In [3]:
manioc_kcal_par_kg = 1250

population_thailande = population_2017.loc[population_2017['pays'] == "Thaïlande", :]

export_manioc_kg = dispo_manioc_thailande['quantite_exportation'].sum() * 1000000

dispo_kcal_export_manioc = manioc_kcal_par_kg * export_manioc_kg

nbr_pers_nourrie_manioc = dispo_kcal_export_manioc / (besoin_kcal_pers_j*365)

proportion_population = nbr_pers_nourrie_manioc * 100 / population_thailande["nombre_habitants"].sum()

print("Quantité export manioc :",export_manioc_kg) 
print("Nombre de personnes pouvant être nourries :", nbr_pers_nourrie_manioc)
print("Proportion de la population :",proportion_population )

NameError: name 'population_2017' is not defined

#### Conclusion
Nous nous donnons comme défi de calculer la proportion de population qui pourrait être nourrie avec les étonnantes exportations de manioc par la Thaïlande. Ainsi nous constatons que la Thailande exporte 25 000 Tonnes de manioc qui pourraient nourrir plus de 38 millions de personnes, soit 55% de la population Thaïlandaise.